In [1]:
class trimVars():
    @staticmethod
    def trimCosTheta_L(df, Btree, pi, ll, q2):
        if int(Btree) == 511:
            if q2 == 1:
                df = df.query('abs(cosTheta_L) < 0.75')
            if q2 == 4:
                df = df.query('abs(cosTheta_L) < 0.95')
        
        if int(Btree) == 521:
            if pi == 'pip':
                if q2 == 1:
                    df = df.query('abs(cosTheta_L) < 0.75')
                elif q2 == 4:
                    df = df.query('abs(cosTheta_L) < 0.95')
            if pi =='pi0':
                if q2 == 1:
                    df = df.query('abs(cosTheta_L) < 0.75')
                elif q2 != 4:
                    df = df.query('abs(cosTheta_L) < 0.9')
        return df
    
    @staticmethod
    def trimCosTheta_K(df, Btree, pi, ll, q2):
        if int(Btree) == 511:
            if q2 == 1:
                df = df.query('cosTheta_K > -0.9')
            elif q2 != 4:
                df = df.query('cosTheta_K > -0.9')
            else:
                df = df.query('abs(cosTheta_K) < 0.95')
                
        if int(Btree) == 521:
            if q2 == 1:
                if pi == 'pip':
                    df = df.query('cosTheta_K > -0.9')
                else:
                    df = df.query('cosTheta_K > 0.9')
            elif q2 != 4:
                if pi == 'pip':
                    df = df.query('cosTheta_K > -0.9')
                elif pi == 'pi0':
                    df = df.query('cosTheta_K > -0.9')
                    
            elif q2 == 4:
                if pi == 'pip':
                    df = df.query('abs(cosTheta_K) > -0.95')
                if pi == 'pi0':
                    df = df.query('abs(cosTheta_K) < 0.9')
        return df
    
    @staticmethod
    def trimmers(df, Btree, pi, ll, q2):
        df = trimVars.trimCosTheta_L(df, Btree, pi, ll, q2)
        df = trimVars.trimCosTheta_K(df, Btree, pi, ll, q2)
        return df

def process_MC_data_combined(file_path_signal, file_path_background, particle_type, Btree, pi, ll, q2,sample_size=None):
    def process_mc_data(file_path, q2,sample_size=None):
        df = uproot.open(f"{file_path}:rec511")
        df = df.arrays(library='pd')
        mc_data = trimVars.trimmers(df, Btree, pi, ll, q2)
        mc_cut = mc_data.query('NNPass==1')
        mc_cut_best_candidate = mc_cut.query('bestCandidate == 1')
        
        if sample_size is not None:
            mc_cut_best_candidate = mc_cut_best_candidate.sample(n=sample_size, random_state=1)  # Adjust random_state as needed
        
        return mc_cut_best_candidate

    mc_cut_best_candidate_signal = process_mc_data(file_path_signal, q2, sample_size)
    mc_cut_best_candidate_background = process_mc_data(file_path_background, q2, sample_size)

    if q2 == 11 or q2 == 12:
        mc_q2cut_background = mc_cut_best_candidate_background.query(f"q2reg == {q2}")

        if particle_type == 'ele':
            cut = f"lm_genMotherPDG == {443 if q2 == 11 else 100443} and lp_genMotherPDG == {443 if q2 == 11 else 100443} and abs(Kst_genMotherPDG) == 511 and Kst_isSignal == 1 and abs(lm_PDG) == 11 and abs(lp_PDG) == 11 and lm_genMotherPDG_1 == Kst_genMotherPDG"
        elif particle_type == 'mu':
            cut = f"lm_genMotherPDG == {443 if q2 == 11 else 100443} and lp_genMotherPDG == {443 if q2 == 11 else 100443} and abs(Kst_genMotherPDG) == 511 and Kst_isSignal == 1 and abs(lm_PDG) == 13 and abs(lp_PDG) == 13 and lm_genMotherPDG_1 == Kst_genMotherPDG"
        else:
            raise ValueError(f"Unknown particle_type: {particle_type}")

        deltaE_range = (-0.15, 0.15)

        mc_signal_q2cut_jpsi_kst_wot_filter = mc_q2cut_background.query(cut)
        mc_signal_q2cut_jpsi_kst = mc_signal_q2cut_jpsi_kst_wot_filter.query('deltaE >= @deltaE_range[0] and deltaE <= @deltaE_range[1]')

        mc_bkg_q2cut_jpsi_kst_wot_filter = mc_q2cut_background.query(f'not ({cut})')
        mc_bkg_q2cut_jpsi_kst = mc_bkg_q2cut_jpsi_kst_wot_filter.query('deltaE >= @deltaE_range[0] and deltaE <= @deltaE_range[1]')
        mc_Continuum_bkg_q2cut_jpsi_kst = mc_bkg_q2cut_jpsi_kst.query('isContinuumEvent==1')
        mc_bkg_q2cut_jpsi_kst = mc_bkg_q2cut_jpsi_kst.query('isContinuumEvent!=1')

        mc_signal_weight_q2cut_jpsi_kst = mc_signal_q2cut_jpsi_kst["PIDCorrectionAndAcceptanceWeight"]
        mc_bkg_weight_q2cut_jpsi_kst = mc_bkg_q2cut_jpsi_kst["PIDCorrectionAndAcceptanceWeight"]
        mc_Continuum_bkg_weight_q2cut_jpsi_kst = mc_Continuum_bkg_q2cut_jpsi_kst["PIDCorrectionAndAcceptanceWeight"]
        mc_bkg_weight_q2cut_jpsi_kst = mc_bkg_q2cut_jpsi_kst["PIDCorrectionAndAcceptanceWeight"]

        # Data
        mc_signal_data = mc_signal_q2cut_jpsi_kst
        mc_bkg_data = mc_bkg_q2cut_jpsi_kst
        mc_Continuum_bkg_data = mc_Continuum_bkg_q2cut_jpsi_kst
        mc_bkg_data = mc_bkg_q2cut_jpsi_kst
        
        # Weight Data
        mc_signal_weight = mc_signal_weight_q2cut_jpsi_kst
        mc_bkg_weight = mc_bkg_weight_q2cut_jpsi_kst
        mc_Continuum_bkg_weight = mc_Continuum_bkg_weight_q2cut_jpsi_kst
        mc_bkg_weight = mc_bkg_weight_q2cut_jpsi_kst

        return mc_signal_data, mc_bkg_data, mc_Continuum_bkg_data, mc_bkg_data, mc_signal_weight, mc_bkg_weight, mc_Continuum_bkg_weight, mc_bkg_weight

    else:
        if q2 in [1, 2, 3, 4]:
            mc_q2cut_signal = mc_cut_best_candidate_signal.query(f"q2reg == {q2}")
            mc_q2cut_background = mc_cut_best_candidate_background.query(f"q2reg == {q2}")
        elif q2 == 0:
            included_q2reg_values = [1, 2, 3, 4]
            excluded_q2reg_values = [11, 12]
            mc_q2cut_signal = mc_cut_best_candidate_signal.query(f'q2reg in {included_q2reg_values} and q2reg not in {excluded_q2reg_values}')
            mc_q2cut_background = mc_cut_best_candidate_background.query(f'q2reg in {included_q2reg_values} and q2reg not in {excluded_q2reg_values}')
        else:
            raise ValueError("Invalid q2 value")

        deltaE_range = (-0.15, 0.15)

        mc_particle_type_signal = mc_q2cut_signal.query('deltaE >= @deltaE_range[0] and deltaE <= @deltaE_range[1]')
        mc_particle_type_only_signal = mc_particle_type_signal.query('isSignal==1')
        mc_particle_type_scf = mc_particle_type_signal.query('isSignal!=1')

        mc_particle_type_bkg = mc_q2cut_background.query('deltaE >= @deltaE_range[0] and deltaE <= @deltaE_range[1]')
        
        mc_particle_type_signal_weight = mc_particle_type_signal["PIDCorrectionAndAcceptanceWeight"]
        mc_particle_type_only_signal_weight = mc_particle_type_only_signal["PIDCorrectionAndAcceptanceWeight"]
        mc_particle_type_scf_weight = mc_particle_type_scf["PIDCorrectionAndAcceptanceWeight"]

        mc_particle_type_bkg_weight = mc_particle_type_bkg["PIDCorrectionAndAcceptanceWeight"]
        
        return (mc_particle_type_signal, mc_particle_type_only_signal, mc_particle_type_scf,
                mc_particle_type_bkg,mc_particle_type_signal_weight, mc_particle_type_only_signal_weight, 
                mc_particle_type_scf_weight,mc_particle_type_bkg_weight)

## Fit ##

In [2]:
def fit(label,particle_type, dataset_name, mc_label_particle_type_dataset_name_data, mc_label_particle_type_dataset_name_weight, model_type,q2reg):
    
    if label == 'Mbc':
        obs_label = zfit.Space(label, limits=(5.20, 5.29))
        
    elif label == 'deltaE':
        obs_label = zfit.Space(label, limits=(-0.15, 0.15))
        
    else:
        raise ValueError(f"Unknown label: {label}")
        
    if particle_type == 'ele':
        mc_dataset_name_data_label = mc_label_particle_type_dataset_name_data
        mc_dataset_name_weight_label = mc_label_particle_type_dataset_name_weight
    elif particle_type == 'mu':
        mc_dataset_name_data_label = mc_label_particle_type_dataset_name_data
        mc_dataset_name_weight_label = mc_label_particle_type_dataset_name_weight

    r = round(np.random.rand(), 4)
    ratios_label_dataset_model_type = None
    
    
    if model_type == 'cb':
        
        # CrystalBall
        mu_particle_type_label_dataset_name_cb = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_cb_{r}", 5.279)
        sigma_particle_type_label_dataset_name_cb = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_cb_{r}", 0.005)
        alpha_particle_type_label_dataset_name_cb = zfit.Parameter(f"alpha_{particle_type}_{label}_{dataset_name}_cb_{r}", 1)
        n_particle_type_label_dataset_name_cb = zfit.Parameter(f"n_{particle_type}_{label}_{dataset_name}_cb_{r}", 2)
        
        
        label_dataset_name_model_type = zfit.pdf.CrystalBall(
                obs=obs_label, mu=mu_particle_type_label_dataset_name_cb, sigma=sigma_particle_type_label_dataset_name_cb,
                alpha=alpha_particle_type_label_dataset_name_cb, n=n_particle_type_label_dataset_name_cb)
   
   
    elif model_type == 'cb_gauss':
        # CrystalBall
        mu_particle_type_label_dataset_name_cb = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_cb_{r}", -0.0)
        sigma_particle_type_label_dataset_name_cb = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_cb_{r}", 0.009)
        alpha_particle_type_label_dataset_name_cb = zfit.Parameter(f"alpha_{particle_type}_{label}_{dataset_name}_cb_{r}", 0.5)
        n_particle_type_label_dataset_name_cb = zfit.Parameter(f"n_{particle_type}_{label}_{dataset_name}_cb_{r}", 1)

        label_dataset_name_cb = zfit.pdf.CrystalBall(
                obs=obs_label, mu=mu_particle_type_label_dataset_name_cb, sigma=sigma_particle_type_label_dataset_name_cb,
                alpha=alpha_particle_type_label_dataset_name_cb, n=n_particle_type_label_dataset_name_cb)

        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", -0.0)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", 0.0096)

        label_dataset_name_gauss = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)

        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}", 0.1)

        label_dataset_name_model_type = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_cb, label_dataset_name_gauss],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)
        
    elif model_type == 'argus_gauss':

        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", 5.289)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}",-120)
        p_particle_type_label_dataset_name_argus= zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", 1)

        label_dataset_name_argus = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", 5.279)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", 0.005)

        label_dataset_name_gauss = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}", 0.5)

        label_dataset_name_model_type = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_argus,label_dataset_name_gauss],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)
        
    elif model_type == 'poly_argus':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}', 0.0)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}', 0.0)

        label_dataset_name_poly = zfit.pdf.Chebyshev(
                obs=obs_label,coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])
        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", 5.289)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}",-120)
        p_particle_type_label_dataset_name_argus= zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", 0.5)

        label_dataset_name_argus = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}", 0.5)

        label_dataset_name_model_type = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_poly, label_dataset_name_argus],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label)
        
    elif model_type == 'argus':
        
        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", 5.289)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}",-120)
        p_particle_type_label_dataset_name_argus= zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", 0.5)

        label_dataset_name_model_type = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
    elif model_type == 'poly':
        
        # Polynomial
        coeff0_ele_label_dataset_name_poly = zfit.Parameter(f'coeff0_ele_{label}_{dataset_name}_poly_{r}', 0.0)
        coeff1_ele_label_dataset_name_poly = zfit.Parameter(f'coeff1_ele_{label}_{dataset_name}_poly_{r}', 0.0)

        label_dataset_name_model_type = zfit.pdf.Chebyshev(
                obs=obs_label,coeffs=[coeff0_ele_label_dataset_name_poly, coeff1_ele_label_dataset_name_poly])

        
        
    elif model_type == 'poly_gauss':
        
        # Polynomial
        coeff0_ele_label_dataset_name_poly = zfit.Parameter(f'coeff0_ele_{label}_{dataset_name}_poly_{r}', 0.0)
        coeff1_ele_label_dataset_name_poly = zfit.Parameter(f'coeff1_ele_{label}_{dataset_name}_poly_{r}', 0.0)

        label_dataset_name_poly = zfit.pdf.Chebyshev(
                obs=obs_label,coeffs=[coeff0_ele_label_dataset_name_poly, coeff1_ele_label_dataset_name_poly])

        # Gaussian
        mu_ele_label_dataset_name_gauss = zfit.Parameter(f"mu_ele_{label}_{dataset_name}_gauss_{r}", 0.0)
        sigma_ele_label_dataset_name_gauss = zfit.Parameter(f"sigma_ele_{label}_{dataset_name}_gauss_{r}", 0.008)

        label_dataset_name_gauss = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_ele_label_dataset_name_gauss, sigma=sigma_ele_label_dataset_name_gauss)
        
        # Fraction
        frac_ele_label_dataset_name_model_type = zfit.Parameter(f"frac_ele_{label}_{dataset_name}_{model_type}_{r}", 0.5)

        label_dataset_name_model_type = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_poly,label_dataset_name_gauss],
            fracs=[frac_ele_label_dataset_name_model_type],
            obs=obs_label)
    
    else:
        raise ValueError(f"Unknown model_type: {model_type}")
        
    
    data_label_dataset_name = zfit.data.Data.from_pandas(
    mc_dataset_name_data_label,
    obs=obs_label,
    weights=mc_dataset_name_weight_label
    )
   
    nll_label_dataset_name = zfit.loss.UnbinnedNLL(model=label_dataset_name_model_type, data=data_label_dataset_name)
    minimizer_label_dataset_name = zfit.minimize.Minuit()
    result_label_dataset_name = minimizer_label_dataset_name.minimize(nll_label_dataset_name)
    
    # Extract and save the parameter values
    parameters_values = {
        "_".join(param.name.split('_')[:-1]): param.numpy() for param in label_dataset_name_model_type.get_params()
    }

    ratios_label_dataset_model_type = pd.DataFrame(parameters_values, index=[0])
    ratios_label_dataset_model_type.to_csv(f"parameter_{particle_type}_{label}_{dataset_name}_{model_type}_{q2reg}.csv", index=False)
    
    return label_dataset_name_model_type, result_label_dataset_name


In [3]:
def plot_with_fit(ax,bins, data, data_weight, fitted_model, data_range, label_data, label_fit, xlabel, ylabel, title):
    # Calculate histogram
    n, bins, _ = ax.hist(data, bins=bins, weights=data_weight, density=False, alpha=0.5, label=label_data)
    bin_centers = (bins[:-1] + bins[1:]) / 2

    # Calculate errors
    bin_weights = []
    for i in range(len(bins) - 1):
        bin_indices = np.where((data >= bins[i]) & (data < bins[i+1]))[0]
        bin_weights.append(data_weight.iloc[bin_indices].tolist())
    errors = [np.sqrt(sum(np.square(w))) for w in bin_weights]

    # De-normalization
    pdf_values = fitted_model.pdf(data_range).numpy() * sum(data_weight) * (bins[1] - bins[0])

    # Plot histogram with error bars and fitted model
    ax.errorbar(bin_centers, n, yerr=errors, fmt='o', color='black', label=f'{label_data} with Error Bars')
    ax.plot(data_range, pdf_values, 'r', label=label_fit)

    # Add labels and title
    ax.legend(fontsize=15)
    ax.set_xlabel(xlabel, fontsize=15)
    ax.set_ylabel(ylabel, fontsize=15)
    ax.set_title(title, fontsize=15)
    return pdf_values

## Yield

In [4]:
def fit_fixed(label,particle_type, dataset_name, mc_label_particle_type_dataset_name_data, mc_label_particle_type_dataset_name_weight, model_type,q2reg):
    
    if label == 'Mbc':
        obs_label = zfit.Space(label, limits=(5.20, 5.29))
        
    elif label == 'deltaE':
        obs_label = zfit.Space(label, limits=(-0.15, 0.15))
        
    else:
        raise ValueError(f"Unknown label: {label}")
        
    if particle_type == 'ele':
        mc_dataset_name_data_label = mc_label_particle_type_dataset_name_data
        mc_dataset_name_weight_label = mc_label_particle_type_dataset_name_weight
    elif particle_type == 'mu':
        mc_dataset_name_data_label = mc_label_particle_type_dataset_name_data
        mc_dataset_name_weight_label = mc_label_particle_type_dataset_name_weight

    r = round(np.random.rand(), 4)
    parameter_df = pd.read_csv(f"parameter_{particle_type}_{label}_{dataset_name}_{model_type}_{q2reg}.csv")
    
    
    if model_type == 'cb':
        
        # CrystalBall
        mu_particle_type_label_dataset_name_cb = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=True)
        sigma_particle_type_label_dataset_name_cb = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        alpha_particle_type_label_dataset_name_cb = zfit.Parameter(f"alpha_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"alpha_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        n_particle_type_label_dataset_name_cb = zfit.Parameter(f"n_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"n_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)

        label_dataset_name_model_type_fixed = zfit.pdf.CrystalBall(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_cb, sigma=sigma_particle_type_label_dataset_name_cb,
            alpha=alpha_particle_type_label_dataset_name_cb, n=n_particle_type_label_dataset_name_cb)
        
    elif model_type == 'cb_gauss':

        # CrystalBall
        mu_particle_type_label_dataset_name_cb = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_cb = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        alpha_particle_type_label_dataset_name_cb = zfit.Parameter(f"alpha_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"alpha_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)
        n_particle_type_label_dataset_name_cb = zfit.Parameter(f"n_{particle_type}_{label}_{dataset_name}_cb_{r}", parameter_df[f"n_{particle_type}_{label}_{dataset_name}_cb"].values[0], floating=False)

        label_dataset_name_cb_fixed = zfit.pdf.CrystalBall(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_cb, sigma=sigma_particle_type_label_dataset_name_cb,
            alpha=alpha_particle_type_label_dataset_name_cb, n=n_particle_type_label_dataset_name_cb)

        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)

        label_dataset_name_gauss_fixed = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)

        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}", 0.5)  

        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_cb_fixed, label_dataset_name_gauss_fixed],
            fracs=frac_particle_type_label_dataset_name_model_type,
            obs=obs_label)



    elif model_type == 'poly_gauss':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f"coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}", parameter_df[f"coeff0_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f"coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}", parameter_df[f"coeff1_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)

        label_dataset_name_poly_fixed = zfit.pdf.Chebyshev(
                obs=obs_label,coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])

        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)

        label_dataset_name_gauss_fixed = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}", 0.5)
        
        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_poly_fixed,label_dataset_name_gauss_fixed],
            fracs=frac_particle_type_label_dataset_name_model_type,
            obs=obs_label)

    
    elif model_type == 'argus_gauss':

        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}",parameter_df[f"m_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}",parameter_df[f"c_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        p_particle_type_label_dataset_name_argus= zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"p_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)

        label_dataset_name_argus_fixed = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
        # Gaussian
        mu_particle_type_label_dataset_name_gauss = zfit.Parameter(f"mu_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"mu_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)
        sigma_particle_type_label_dataset_name_gauss = zfit.Parameter(f"sigma_{particle_type}_{label}_{dataset_name}_gauss_{r}", parameter_df[f"sigma_{particle_type}_{label}_{dataset_name}_gauss"].values[0], floating=False)

        label_dataset_name_gauss_fixed = zfit.pdf.Gauss(
            obs=obs_label, mu=mu_particle_type_label_dataset_name_gauss, sigma=sigma_particle_type_label_dataset_name_gauss)
        
        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}", 0.5)

        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_argus_fixed,label_dataset_name_gauss_fixed],
            fracs=frac_particle_type_label_dataset_name_model_type,
            obs=obs_label)
        
    elif model_type == 'argus':
        
        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", 5.28819 )
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"c_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        p_particle_type_label_dataset_name_argus = zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"p_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)

        label_dataset_name_model_type_fixed = zphysics.pdf.Argus(
            obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)
        
    
        
    elif model_type == 'poly':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}', 0.0)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff1_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)

        label_dataset_name_model_type_fixed = zfit.pdf.Chebyshev(obs=obs_label, coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])


  
    elif model_type == 'poly_argus':
        
        # Polynomial
        coeff0_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff0_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff0_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)
        coeff1_particle_type_label_dataset_name_poly = zfit.Parameter(f'coeff1_{particle_type}_{label}_{dataset_name}_poly_{r}', parameter_df[f"coeff1_{particle_type}_{label}_{dataset_name}_poly"].values[0], floating=False)

        label_dataset_name_poly_fixed = zfit.pdf.Chebyshev(obs=obs_label, coeffs=[coeff0_particle_type_label_dataset_name_poly, coeff1_particle_type_label_dataset_name_poly])
        # Argus
        m_particle_type_label_dataset_name_argus = zfit.Parameter(f"m_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"m_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        c_particle_type_label_dataset_name_argus = zfit.Parameter(f"c_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"c_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)
        p_particle_type_label_dataset_name_argus = zfit.Parameter(f"p_{particle_type}_{label}_{dataset_name}_argus_{r}", parameter_df[f"p_{particle_type}_{label}_{dataset_name}_argus"].values[0], floating=False)

        label_dataset_name_argus_fixed = zphysics.pdf.Argus(obs=obs_label, m0=m_particle_type_label_dataset_name_argus, c=c_particle_type_label_dataset_name_argus, p=p_particle_type_label_dataset_name_argus)

        # Fraction
        frac_particle_type_label_dataset_name_model_type = zfit.Parameter(f"frac_{particle_type}_{label}_{dataset_name}_{model_type}_{r}", 0.5)

        label_dataset_name_model_type_fixed = zfit.pdf.SumPDF(
            pdfs=[label_dataset_name_poly_fixed, label_dataset_name_argus_fixed],
            fracs=[frac_particle_type_label_dataset_name_model_type],
            obs=obs_label
        )
            
    else:
        raise ValueError(f"Unknown model_type: {model_type}")
        
    return label_dataset_name_model_type_fixed
    

     
        
'''   
    data_label_dataset_name = zfit.data.Data.from_pandas(mc_label_particle_type_dataset_name_data, obs=obs_label, weights=mc_label_particle_type_dataset_name_weight)
    nll_label_dataset_name = zfit.loss.UnbinnedNLL(model=label_dataset_name_model_type_fixed, data=data_label_dataset_name)
    minimizer_label_dataset_name = zfit.minimize.Minuit()
    result_label_dataset_name = minimizer_label_dataset_name.minimize(nll_label_dataset_name)
        
    return label_dataset_name_model_type_fixed, result_label_dataset_name
    


'''     

'   \n    data_label_dataset_name = zfit.data.Data.from_pandas(mc_label_particle_type_dataset_name_data, obs=obs_label, weights=mc_label_particle_type_dataset_name_weight)\n    nll_label_dataset_name = zfit.loss.UnbinnedNLL(model=label_dataset_name_model_type_fixed, data=data_label_dataset_name)\n    minimizer_label_dataset_name = zfit.minimize.Minuit()\n    result_label_dataset_name = minimizer_label_dataset_name.minimize(nll_label_dataset_name)\n        \n    return label_dataset_name_model_type_fixed, result_label_dataset_name\n    \n\n\n'

In [5]:
def fit_extended_model(Mbc_only_signal_pdf,Mbc_scf_pdf, deltaE_only_signal_pdf,deltaE_scf_pdf, Mbc_bkg_pdf,deltaE_bkg_pdf,data,weights):
    
    obs_Mbc = zfit.Space('Mbc', limits=(5.20, 5.29))
    obs_deltaE = zfit.Space('deltaE', limits=(-0.15, 0.15))
    
    combined_obs = obs_Mbc*obs_deltaE
    
    # Random number for parameter names
    r = np.round(np.random.rand(), decimals=4)
    
    # Signal model
    

    Mbc_signal_pdf = Mbc_only_signal_pdf
    deltaE_signal_pdf = deltaE_only_signal_pdf
    
    signal_product_pdf = zfit.pdf.ProductPDF([Mbc_signal_pdf, deltaE_signal_pdf])
    signal_yield = zfit.Parameter(f"signal_yield_{r}", 1000)
    signal_extended = signal_product_pdf.create_extended(signal_yield)

    # Background model
    frac_Mbc_bkg = zfit.Parameter(f'frac_Mbc_bkg_{r}', 0.5)
    frac_deltaE_bkg = zfit.Parameter(f'frac_deltaE_bkg_{r}', 0.5)

    
    Mbc_background_pdf = zfit.pdf.SumPDF([Mbc_scf_pdf,Mbc_bkg_pdf], fracs=[frac_Mbc_bkg])
    deltaE_background_pdf = zfit.pdf.SumPDF([deltaE_scf_pdf,deltaE_bkg_pdf], fracs=[frac_deltaE_bkg])

    background_product_pdf = zfit.pdf.ProductPDF([Mbc_background_pdf, deltaE_background_pdf])
    background_yield = zfit.Parameter(f"background_yield_{r}", 1000)
    background_extended = background_product_pdf.create_extended(background_yield)

    # Total model sum
    total_model = zfit.pdf.SumPDF([signal_extended, background_extended])
    
    # Create a joint data for Mbc and deltaE
    combined_data = zfit.data.Data.from_pandas(data, obs=combined_obs, weights=weights)

    # Create an extended NLL for the model and data
    nll = zfit.loss.ExtendedUnbinnedNLL(model=total_model, data=combined_data)

    # Minimize the NLL
    minimizer = zfit.minimize.Minuit()
    result = minimizer.minimize(nll)
    
    return Mbc_signal_pdf,deltaE_signal_pdf,Mbc_background_pdf,deltaE_background_pdf,signal_extended, background_extended, total_model, result




In [6]:
'''
signal_yield_value = result.params[signal_extended.get_yield()]['value']
background_yield_value = result.params[background_extended.get_yield()]['value']

# Round the values
signal_event_fit = round(signal_yield_value, 3)
bkg_event_fit = round(background_yield_value, 3)


mc_ele_signal_weight = mc_ele_only_signal_weight
mc_ele_bkg_weight_mc = pd.concat([mc_ele_scf_weight,mc_ele_bkg_weight])

mc_ele_signal_MC = round(sum(mc_ele_signal_weight), 3)
mc_ele_bkg_MC = round(sum(mc_ele_bkg_weight_mc), 3)


# Print the results
print("Results from Fit:")
print("Signal Events =", signal_event_fit)
print("Background Events =", bkg_event_fit)
print()

print("Results from MC Data:")
print("Signal Events in MC Data =", mc_ele_signal_MC)
print("Background Events in MC Data =", mc_ele_bkg_MC)
print()

errors_sig_events = mc_ele_signal_MC - signal_event_fit
print("Signal Events different =", round(errors_sig_events, 3))

errors_bkg_events = mc_ele_bkg_MC - bkg_event_fit
print("Background Events different=", round(errors_bkg_events, 3))
'''

'\nsignal_yield_value = result.params[signal_extended.get_yield()][\'value\']\nbackground_yield_value = result.params[background_extended.get_yield()][\'value\']\n\n# Round the values\nsignal_event_fit = round(signal_yield_value, 3)\nbkg_event_fit = round(background_yield_value, 3)\n\n\nmc_ele_signal_weight = mc_ele_only_signal_weight\nmc_ele_bkg_weight_mc = pd.concat([mc_ele_scf_weight,mc_ele_bkg_weight])\n\nmc_ele_signal_MC = round(sum(mc_ele_signal_weight), 3)\nmc_ele_bkg_MC = round(sum(mc_ele_bkg_weight_mc), 3)\n\n\n# Print the results\nprint("Results from Fit:")\nprint("Signal Events =", signal_event_fit)\nprint("Background Events =", bkg_event_fit)\nprint()\n\nprint("Results from MC Data:")\nprint("Signal Events in MC Data =", mc_ele_signal_MC)\nprint("Background Events in MC Data =", mc_ele_bkg_MC)\nprint()\n\nerrors_sig_events = mc_ele_signal_MC - signal_event_fit\nprint("Signal Events different =", round(errors_sig_events, 3))\n\nerrors_bkg_events = mc_ele_bkg_MC - bkg_event_f

## with fit data ##

In [7]:
'''
import numpy as np
from uncertainties import ufloat

# Extract signal yield and its error from fit result
signal_yield = result.params[signal_extended.get_yield()]['value']
signal_yield_errors = result.params[signal_extended.get_yield()]['errors']

# Calculate the standard deviation from the given error range
signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2


# Create an ufloat for the signal yield with its uncertainty
signal_yield_with_uncertainty = ufloat(signal_yield, signal_yield_error)

# Adjust the signal yield for the percentage change (18.53%)
adjusted_signal_yield_with_uncertainty = signal_yield_with_uncertainty * (1 + 0.0747)

# Constants
cross_section_nb = 0.5346e-9  # Cross-section in nanobarns
luminosity_nb = 368.4e+15  # Luminosity in femtobarns

# Calculate the number of B mesons
N_bb_mesons = cross_section_nb * (4 * luminosity_nb)

# Branching ratio calculation
Y_B0B0 = ufloat(48.6, 0.6) / 100
BR_MC = adjusted_signal_yield_with_uncertainty / ((2 * N_bb_mesons) * Y_B0B0)

# PDG value for comparison
central_value = 1.03e-6
average_uncertainty = (0.19e-6 + 0.17e-6) / 2
BR_PDG = ufloat(central_value, average_uncertainty)

# Print results
print("B _to_ee_and_K*_for_electron_Branching_ratio:", BR_MC)
print("B _to_ee_and_K*_Branching_ratio_from_PDG:", BR_PDG)
'''

'\nimport numpy as np\nfrom uncertainties import ufloat\n\n# Extract signal yield and its error from fit result\nsignal_yield = result.params[signal_extended.get_yield()][\'value\']\nsignal_yield_errors = result.params[signal_extended.get_yield()][\'errors\']\n\n# Calculate the standard deviation from the given error range\nsignal_yield_error = (signal_yield_errors[\'upper\'] - signal_yield_errors[\'lower\']) / 2\n\n\n# Create an ufloat for the signal yield with its uncertainty\nsignal_yield_with_uncertainty = ufloat(signal_yield, signal_yield_error)\n\n# Adjust the signal yield for the percentage change (18.53%)\nadjusted_signal_yield_with_uncertainty = signal_yield_with_uncertainty * (1 + 0.0747)\n\n# Constants\ncross_section_nb = 0.5346e-9  # Cross-section in nanobarns\nluminosity_nb = 368.4e+15  # Luminosity in femtobarns\n\n# Calculate the number of B mesons\nN_bb_mesons = cross_section_nb * (4 * luminosity_nb)\n\n# Branching ratio calculation\nY_B0B0 = ufloat(48.6, 0.6) / 100\nBR

## BR with MC weights ##

In [8]:
'''

mc_ele_signal_MC = sum(mc_ele_signal_weight)

# Adjust the signal yield for the percentage change (18.53%)
adjusted_signal_yield_with_uncertainty = mc_ele_signal_MC * (1 + 0.1853)

# Constants
cross_section_nb = 0.5346e-9  # Cross-section in nanobarns
luminosity_nb = 368.4e+15  # Luminosity in femtobarns

# Calculate the number of B mesons
N_bb_mesons = cross_section_nb * (4 * luminosity_nb)

# Branching ratio calculation
Y_B0B0 = ufloat(48.6, 0.6) / 100
BR_MC = adjusted_signal_yield_with_uncertainty / ((2 * N_bb_mesons) * Y_B0B0)

# PDG value for comparison
central_value = 1.03e-6
average_uncertainty = (0.19e-6 + 0.17e-6) / 2
BR_PDG = ufloat(central_value, average_uncertainty)

# Print results
print("B _to_ee_and_K*_for_electron_Branching_ratio:", BR_MC)
print("B _to_ee_and_K*_Branching_ratio_from_PDG:", BR_PDG)

'''

'\n\nmc_ele_signal_MC = sum(mc_ele_signal_weight)\n\n# Adjust the signal yield for the percentage change (18.53%)\nadjusted_signal_yield_with_uncertainty = mc_ele_signal_MC * (1 + 0.1853)\n\n# Constants\ncross_section_nb = 0.5346e-9  # Cross-section in nanobarns\nluminosity_nb = 368.4e+15  # Luminosity in femtobarns\n\n# Calculate the number of B mesons\nN_bb_mesons = cross_section_nb * (4 * luminosity_nb)\n\n# Branching ratio calculation\nY_B0B0 = ufloat(48.6, 0.6) / 100\nBR_MC = adjusted_signal_yield_with_uncertainty / ((2 * N_bb_mesons) * Y_B0B0)\n\n# PDG value for comparison\ncentral_value = 1.03e-6\naverage_uncertainty = (0.19e-6 + 0.17e-6) / 2\nBR_PDG = ufloat(central_value, average_uncertainty)\n\n# Print results\nprint("B _to_ee_and_K*_for_electron_Branching_ratio:", BR_MC)\nprint("B _to_ee_and_K*_Branching_ratio_from_PDG:", BR_PDG)\n\n'

In [9]:
def plot_variable(ax, ax_pull, variable, data_only_signal, data_scf, bkg, weight_only_signal, weight_scf, weight_bkg, 
                  signal_pdf, background_pdf, label_signal, label_background, xaxis):
    
    signal_df = data_only_signal[variable]
    bkg_df = pd.concat([data_scf[variable], bkg[variable]])
    
    df_total = pd.concat([signal_df, bkg_df])
    
    signal_weight = weight_only_signal
    bkg_weight = pd.concat([weight_scf, weight_bkg])
    
    total_weight = pd.concat([signal_weight, bkg_weight])

    
    # Define bin edges and x_fit
    if variable == "Mbc":
        bin_edges = np.linspace(5.20, 5.2901, 100)
        x_fit = np.linspace(5.20, 5.2901, 100)
    elif variable == "deltaE":
        bin_edges = np.linspace(-0.15, 0.15, 100)
        x_fit = np.linspace(-0.15, 0.15, 100)

    # Plot the background component
    y_background_fit = background_pdf.pdf(x_fit) * sum(bkg_weight) * np.diff(bin_edges)[0]
    ax.plot(x_fit, y_background_fit, label=label_background, linestyle="--", color="blue", linewidth=2)

    # Plot the signal component
    y_signal_fit = signal_pdf.pdf(x_fit) * sum(signal_weight) * np.diff(bin_edges)[0]
    ax.plot(x_fit, y_signal_fit, label=label_signal, linestyle="-", color="red", linewidth=2)

    # Plot the total fit
    n, bins, _ = ax.hist(df_total, bins=bin_edges, weights=total_weight, density=False, color='white', alpha=0.3, edgecolor='black')
    bin_centers = (bins[:-1] + bins[1:]) / 2
    
    # Calculate errors
    bin_weights = []
    for i in range(len(bin_edges) - 1):
        bin_indices = np.where((df_total >= bins[i]) & (df_total < bins[i+1]))[0]
        bin_weights.append(total_weight.iloc[bin_indices].tolist())
    errors = [np.sqrt(sum(np.square(w))) for w in bin_weights]

    # Plot histogram with error bars and fitted model
    ax.errorbar(bin_centers, n, yerr=errors, fmt='o', markersize=5, color='black', label='Data', capsize=3)

    # Plot the total fit with error bars
    y_total_fit = y_signal_fit + y_background_fit
    y_total_fit_binned = np.interp(bin_centers, x_fit, y_total_fit)
    ax.plot(x_fit, y_total_fit, label="Fit", color="black", linewidth=2)
    
    # Calculate pull and plot
    pulls = (n - y_total_fit_binned) / errors
    ax_pull.bar(bin_centers, pulls, width=np.diff(bin_edges), align='center', alpha=0.5, color='gray')
    ax_pull.axhline(0, color='black', linewidth=1)
    ax_pull.set_xlabel(xaxis, fontsize=14)
    ax_pull.set_ylabel('Pull', fontsize=14)
    ax_pull.grid(True, linestyle='--', alpha=0.7)

    # Add labels and legend
    ax.set_ylabel('Counts', fontsize=14)
    ax.legend(fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.7)

